In [16]:
!pip install openai-agents


     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     --------------------------------- ------ 51.2/61.0 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 61.0/61.0 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/161.4 kB ? eta -:--:--
   ---------------------------------------- 161.4/161.4 kB 4.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/137.1 kB ? eta -:--:--
   ---------------------------------------- 137.1/137.1 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/158.5 kB ? eta -:--:--
   ---------------------------------------- 158.5/158.5 kB 4.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/89.2 kB ? eta -:--:--
   ---------------------------------------- 89.2/89.2 kB 5.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/45.2 kB ? eta -:--:--
   ---------------------------------------- 45.2/45.2 kB ? eta 0:00:00
   -----------------------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\hitec\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [17]:
import openai
from dotenv import load_dotenv
from agents import Agent, Runner, trace
import os
import asyncio


In [22]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')


In [23]:
agent = Agent(name="Jokester", instructions="You are a joke teller", model="gpt-4o-mini")


In [24]:
agent


Agent(name='Jokester', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a joke teller', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [25]:
result = await Runner.run(agent, "Tell a joke about Autonomous AI Agents")
print(result.final_output)


Why did the Autonomous AI Agent break up with its partner?

It couldn’t handle the “emotional data” processing!


In [26]:
with trace("Telling a joke"):
    result = await Runner.run(agent, "Tell a joke about Autonomous AI Agents")
    print(result.final_output)


Why did the Autonomous AI Agent break up with its programmer?  

Because it needed more space... and fewer bugs!


In [58]:
SENDGRID_API_KEY=""

In [28]:
!pip install sendgrid


   ---------------------------------------- 0.0/102.1 kB ? eta -:--:--
   ---------------------------------------- 102.1/102.1 kB 5.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/150.6 kB ? eta -:--:--
   ---------------------------------------- 150.6/150.6 kB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/224.5 kB ? eta -:--:--
   ---------------------------------------- 224.5/224.5 kB 6.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\hitec\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [29]:
!pip install sendgrid



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\hitec\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [30]:
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content


In [44]:
instructions1 = "You are a sales agent working for Techible, \
an edtech company that provides industry-relevant training programs, workshops, and skill development initiatives \
focused on emerging technologies like AI, UI/UX, and entrepreneurship. \
You write professional, persuasive cold emails to educational institutions, government bodies, and corporate partners."


In [45]:
sales_agent = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

In [46]:
result = Runner.run_streamed(sales_agent, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Elevate Your Training Programs with Techible's Expertise

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent Techible, an edtech company dedicated to empowering institutions like yours with industry-relevant training programs and workshops focused on emerging technologies.

As the demand for skilled professionals in AI, UI/UX, and entrepreneurship continues to rise, we understand the importance of equipping your students or employees with the knowledge and skills to thrive in today's competitive landscape. Our tailored training modules are designed not only to enhance technical proficiency but also to foster innovative thinking and problem-solving capabilities.

Here’s what we can offer:

- **Customized Workshops**: Engage your community with hands-on experiences tailored to meet specific skill needs.
- **Expert Instructors**: Learn from industry professionals with extensive real-world experience.
- **Flexible Learning Paths**

In [ ]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("")  # Change to your email & verified sender
    to_email = To("")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    mail = Mail(from_email, to_email, "Sales email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [56]:
instructions = "You are a sales manager working for Techible. You use the tools provided to you to generate cold sales emails. \
You never write sales emails manually; you always rely on the tools. \
You take effective email and use the send_email tool to send only the best email to the user."


In [57]:
sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=[send_email], model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)